In [ ]:
import selenium
from selenium import webdriver
import time
import subprocess
import math
import os
import random
import tempfile
from pyvirtualdisplay import Display

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.keys import Keys



In [ ]:
class Film(object):
    def __init__(self, height, width, filename):
        self.offset = 80
        display = Display(visible=1, size=(width,height+self.offset))
        display.start()
        self.display_id = display.cmd[-1]
        self.height = height
        self.width = width
        chromeOptions = webdriver.ChromeOptions()
        #chromeOptions.add_arguments("--kiosk")
        #chromeOptions.add_argument("")
        #chromeOptions.add_argument("--disable-infobars --start-fullscreen --kiosk".split())
        chromeOptions.add_argument("--disable-infobars")
        
        self.driver = webdriver.Chrome(chrome_options=chromeOptions)
        
        self.driver.set_window_size(width, height+self.offset)
        #self.tmpdir = tempfile.mkdtemp()
        self.filename = filename
        self.sequence_position = 0
        
    def get_view_height(self):
        return int(self.driver.execute_script("return window.innerHeight"))
    
    def render_frames(self, top, bottom):
        dst = f"hej-{str(self.sequence_position).zfill(2)}.mp4"
        
        ffmpegcmd = f"""ffmpeg -y -f x11grab -r 60 -s {self.width}x{self.height+self.offset} -i {self.display_id} -an -vcodec libx264 -vf crop={self.width}:{self.height}:0:{self.offset} {dst}"""
        print(ffmpegcmd)
        p = subprocess.Popen(ffmpegcmd.split())
        
        frames = []
        for e,h in enumerate(range(top,bottom,1),start = self.sequence_position):
            self.driver.execute_script(f"window.scrollTo(0,{h});")
            #dst = f"{self.tmpdir}/{str(e).zfill(5)}.png"
            #self.driver.save_screenshot(dst)
            #frames.append(self.driver.get_screenshot_as_png())
        self.sequence_position += 1
        p.terminate()
    
    def single(self, url, selector):
        self.driver.get(url)
        el = self.driver.find_element_by_css_selector(selector)
        
        top = el.location['y']
        bottom = int(el.location["y"]+el.size['height']-self.get_view_height())
        self.render_frames(top,bottom)
        
    def linear(self, url, selector_from, selector_to ):
        self.driver.get(url)
        el_top = self.driver.find_element_by_css_selector(selector_from)
        el_bottom = self.driver.find_element_by_css_selector(selector_to)
        
        top = el_top.location['y']
        bottom = int(el_bottom.location["y"]+el_bottom.size['height']-self.get_view_height())
        
        self.render_frames(top,bottom)
            
    def render(self):
        cmd = f"""ffmpeg -r 60 -i {self.tmpdir}/%05d.png -c:v libx264 -r 60 -pix_fmt yuv420p {self.filename} """
        subprocess.check_call(cmd, shell=True)

In [ ]:
story = {
    'height': 720,
    'width': 1280,
    'shots': [
        {
        'url':"https://www.svt.se/nyheter/inrikes/fyra-atalas-i-allra-harvan"
        'script': [
            {'type': 'single','params':{'selector':'.nyh_article',,}},
            #{'type': 'linear','params':{'selector_from':'#main-content','selector_to':"#comments",'url':"https://svenska.yle.fi/artikel/2018/03/30/din-generation-hur-har-det-gatt-for-er-som-ar-fodda-samma-ar",}},
        ]
        }
    ]
}

film = Film(shot["height"], shot["width"], "test_class.mp4")
for s in shot["script"]:
    fn = getattr(film, s["type"])
    fn(**s["params"])
#film.render()


In [ ]:
shot = {
    'height': 720,
    'width': 1280,
    'script': [
        {'type': 'linear','params':{'selector_from':'h2','selector_to':".checkboxes",'url':"https://www.svt.se/special/hur-lika-ar-kandidaterna-sina-partier/",}},
        
    ]
}
film = Film(shot["height"], shot["width"], "test_hur_lika_ar_kandidaterna_jump.mp4")
for s in shot["script"]:
    fn = getattr(film, s["type"])
    #fn(**s["params"])
#film.render()

In [ ]:
film.driver.find_element_by_css_selector("body").send_keys(Keys.F11)